<a href="https://colab.research.google.com/github/kaleynguyen/ds-security/blob/main/Copy_of_Week_1_Project_(originalcopy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 1 Project

Welcome to our week 1 project! This notebook will help you get started to explore the Alpha Bank website at https://class.disfraud.com/.

For reference, the project instructions can be found at https://corise.com/course/data-science-for-security/module/week-1-project. 

## Explore the Alpha Bank website

The first step is to visit the Alpha Bank website on your browser, and inspect the website via Developer Tools.

As a refresher, here is a [link to the course material (videos)](https://corise.com/course/data-science-for-security/module/using-browser-developer-tools) on using Developer Tools.

Try creating several accounts. Here are some guiding questions:

* What do you notice, for each request and response? What is in the request and response JSON?
* What feedback does the website provide? Are there any useful messages that might help you constrain your "attack space"?
* What is the format of each username? What are the password requirements/restrictions?
* Are there any endpoints that aren't protected?

## Perform a simple login to your own account

Here is some code that uses Python requests to make a POST request to the login API endpoint.

To obtain url and post_body, first inspect the request sent by your browser when you (manually) login to your own account.

How do you think the token is generated?

In [ ]:
import requests

# Update URL and post_body below
url = ""
post_body = {}
response = requests.post(url, json=post_body)

response.json()

### A note on Python requests

If you use Python requests, and need to pass along a dictionary in your request, remember to either specify a `content-type` header or use the json argument.

As an example, both of these requests won't work properly (you'll likely get a server error):

```
post_body = {"param1": "value1", "param2": "value2"}
response = requests.post(url, data=post_body)
response = requests.post(url, data=json.dumps(post_body))
```

You'll need to do either this:

```
post_body = {"param1": "value1", "param2": "value2"}
headers = {
    "Content-type": "application/json"
}
response = requests.post(url, data=json.dumps(post_body), headers=headers)
```

Or this (which I prefer, as it is cleaner):

```
post_body = {"param1": "value1", "param2": "value2"}
response = requests.post(url, json=post_body)
```

In the latter example, Python automatically specifies the right header in your request, so you don't have to worry about it.

# Infer pattern(s) from security tokens

Let's try to fetch a larger sample of tokens and see if we can infer some common pattern.

In [ ]:
url = ""
post_body = {}

tokens = []
for i in range(10):
    response = requests.post(url, json=post_body)
    response_json = response.json()
    token = response_json["new_token"]
    tokens.append(token)

tokens

## Generating tokens programmatically

Now that you understand what a token is, can you write a function to generate your own token?

In [ ]:
def generate_token():
    token = ""
    return token

Hint: If you get stuck on this for more than 10 minutes, read the ["Detour"](https://corise.com/course/data-science-for-security/module/web-apps-101) section in these course notes carefully.
 
If you still need help, reach out on Slack!

## Taking over accounts

Since you are now able to generate a valid security token anytime, it should be trivial for you to take over some accounts.

Here are some tips to get you started.

### Account numbers
What is the range of valid account numbers? _(Hint: if you don't know, or can't remember, try creating a new account. Was there a helpful notice somewhere?)_

### Passwords
What is the password format? _(Hint: if you aren't sure, go to the 'create account' page and walk through the process of choosing a password!)_

It is definitely possible to brute force these passwords... but is there a better way?

Some passwords are more common than others. Based on the password format, what do you think are some likely passwords? _(Hint: Google is your friend. A lot has been written about this.)_

---

**Helpful Python tip:**
If you need to pad your number to a certain fixed length, try the `zfill()` method.


In [ ]:
# Define the list of accounts
accounts = []

# Define the potential password range
passwords = []


# Setting up and running your attack script

Now that you have your target accounts and potential passwords set up, you should be able to programmatically find out which accounts can be taken over. Good thing Alpha Bank has no 2FA in place!

Executing this "account discovery" process should be a simple matter of running a set of for loops.

Please be kind to the bank and to your classmates: try not to bring the bank's website down!

---

**Helpful Python tip:**

Insert some random amount of time before every consecutive request! `sleep()` is your friend. If you don't observe this, you may get blocked (by your ISP / by the server / by anything in between).

In [ ]:
import json

def get_account_balance(account_number, password):
    """Attempt a login to account_number with password, to discover the account balance."""
    login_success = False
    account_balance = 0
    token = generate_token()

    # your code here
    
    return login_success, account_balance

def discover_accounts(accounts, passwords):
    """Attack Alpha Bank's login endpoint to discover whether accounts exist."""

    valid_accounts = []

    for account in accounts:
        # Implement some stop condition here.
        # Don't be greedy! There is no need to discover ALL possible accounts. 
        # As long as you think you'll earn enough from all the account balances you've discovered...

        cracked = False
        for password in passwords:
            cracked, balance = get_account_balance(account, password)
            if cracked:
                print(f"Bingo! One more account: {account}. Password: {password}. Balance: ${balance:,}")
                valid_accounts.push({"account": account, "password": password, "balance": balance})
                continue
    
    return valid_accounts

valid_accounts = discover_accounts(accounts, passwords)

# Better save your valid accounts to a text file in case this notebook crashes!
with open("gold_mine.txt", "w") as f:
    f.write(json.dumps(valid_accounts))

# Transferring money

Great! Now you have (hopefully) gathered a number of accounts. You'll need to figure out how to transfer money from those accounts to your own.

The best way to test this is to create two or more accounts of your own, and transfer money between them. (Don't worry, Alpha Bank does not charge transfer fees! That may change in the future though.)

As you did for the login, watch the JSON requests and responses for money transfer.

---

**Helpful Python tip:**

Since we are making multiple calls, it will be easier to use a requests session. This helps you manage cookies easily between requests in the same session.

See documentation [here](https://requests.readthedocs.io/en/latest/user/advanced/). I've included some boilerplate code below.

In [ ]:
import requests
import json

def login_and_transfer_money(source_account, target_account):
    """Performs a login to source_account and transfers money to target_account."""
    login_url = ""
    money_transfer_url = ""
    transferred_amount = 0
    updated_target_account_balance = None

    with requests.Session() as s:
        # This initiates the requests within a session. The benefit is that any cookies are captured automatically.
        login_body = {
            "...": "..."
        }
        r = s.post(login_url, json=login_body)
        
        # Response headers are available in r.headers.
        # To inspect them, do something like:
        #   print(json.dumps(dict(r.headers), indent=4))
        
        # Request cookies are found in a "requests cookie jar", with:
        #   r.cookies

        # Call money transfer endpoint
        # Do you want to transfer all of the money, or leave some money behind?
        money_transfer_body = {
            "...": "..."
        }
        r2 = s.post(money_transfer_url, json=money_transfer_body)

        # Check the response that you receive, and infer if your money has been transferred successfully.
        # You can also get your updated balance from your response.

    return transferred_amount, updated_target_account_balance

# Time to get rich!

Now that you've managed to login and perform money transfers automatically, let's see how much money you can get into your own account!

In [ ]:
def get_rich_quick(compromised_accounts, my_account):
    """The final battle."""
    total_amount_transferred = 0
    for account in compromised_accounts:
        transferred_amount, updated_target_account_balance = login_and_transfer_money(account, my_account)
        print(f"Transferred {transferred_amount} from {account}. My new balance: ${updated_target_account_balance:,}")
        total_amount_transferred += transferred_amount
    print(f"Final balance: ${updated_target_account_balance:,}")
    print(f"Total $$ swindled: ${total_amount_transferred:,}")